In [ ]:
import numpy as np
from matplotlib import image
import matplotlib.pyplot as plt
import matplotlib.colors as clr
from scipy.fftpack import dct
from scipy.fftpack import idct
import PIL as p

In [ ]:
"""
Returns an array of type:
 - [x, y] for grayscale images
 - [x, y, [R, G, B]] for RGB images
 - [x, y, [R, G, B, A]] for RGBA images
"""
def read_image(filename):
    img = image.imread(filename)
    return img

*3.2 função para criar colormap entre duas cores*

In [ ]:
"""
Funcao para criar um colormap;
Parametros:
            1.cmin-> tuplo com valores (r,g,b) minimos
            2.cmax-> tuplo com valores (r,g,b) maximos
Devolve o objeto correspondente ao colormap
"""
def create_colormap(cmin : tuple[float], cmax: tuple[float]):
    return clr.LinearSegmentedColormap.from_list('', [cmin, cmax], N=256)

*3.3 Função para visualizar a imagem com um certo colormap*

In [ ]:
"""
Função para mostrar uma imagem. Aceita um colormap definido pelo utilizador ou os do matplotlib
"""
    
def show_image(img, colormap = None):    
    plt.figure(figsize=(8,8))
    
    # Imagens com apenas uma coponenete: R, G, B ou Grayscale
    if len(img.shape) == 2:
        plt.imshow(img, cmap = colormap)
    else:
        if colormap != None:
            new_img = img[:, :, 0]
            plt.imshow(new_img, cmap = colormap)
        else:
            plt.imshow(img)

    plt.axis('off')
    plt.show()

> # Semana 1
Na diretória imagens estão presentes as imagens jpeg com baixa, média e alta qualidade, de acordo com o nome das pastas
> - Análise da compressão das várias imagens em jpeg
> - Converter a imagem de RGB para YCbCr
> - *Padding* da imagen para ficar um múltiplo de 16x16
 


## 3.1 Leitura da imagem

In [ ]:
#img = read_image('imagens/peppers.bmp')
img = read_image('imagens/barn_mountains.bmp')
#img = read_image('imagens/logo.bmp')

show_image(img)


### Tamanho dos ficheiros .bmp

|        | Barn | Peppers | Logo |
|:------:|:----:|:-------:|:----:|
|  size  |   356.5KB  |    589.9KB   |   421.6KB  |


<br>
### Tamanho dos ficheiros após compressão para JPEG

|        | Barn | Peppers | Logo |
|:------:|:----:|:-------:|:----:|
|   Low  |   43.4KB  |    35.2KB    |   21.9KB  |
| Medium |   51.5KB  |    41.3KB    |   23.1KB  |
|  High  |   67.5KB  |    57.7KB    |   27.3KB  |

<br>
### Rácio de compressão

|        | Barn | Peppers | Logo |
|:------:|:----:|:-------:|:----:|
|   Low  |   8.2:1  |    16.7:1    |   19.3:1  |
| Medium |   6.9:1  |    14.3:1    |   18.3:1  |
|  High  |   5.2:1  |    10.2:1    |   15.4:1  |

<br>
### Diferenças subjetivas jpeg/bmp

|        | Barn                 | Peppers | Logo |
|:------:|:------------------:|:--------------:|:-----------------:|
|   Low  | não muito evidente | muito evidente     | evidente      |
| Medium | nada evidente      | não muito evidente | evidente      |
|  High  | nada evidente      | nada evidente      | nada evidente |

## Comparação dos resultados - TODO
A compressão aparenta ser mais evidente em imagens cujos pixels adjacentes têm uma cor bastante semelhante, sendo particularmente notável em casos em que as cores apresentam brilho elevado

In [ ]:
low = image.imread('imagens/Low/peppers.jpg')
low.shape

In [ ]:
medium = image.imread('imagens/Medium/peppers.jpg')
medium.shape

In [ ]:
# ColorMaps
cm_gray = clr.LinearSegmentedColormap.from_list('gray', [(0,0,0), (1, 1, 1)], N = 256)
cm_red = clr.LinearSegmentedColormap.from_list('red', [(0,0,0), (1, 0, 0)], N = 256)
cm_green = clr.LinearSegmentedColormap.from_list('green', [(0,0,0), (0, 1, 0)], N = 256)
cm_blue = clr.LinearSegmentedColormap.from_list('blue', [(0,0,0), (0, 0, 1)], N = 256)


# Adaptado de https://jakevdp.github.io/PythonDataScienceHandbook/04.07-customizing-colorbars.html
cmap = plt.cm.get_cmap(cm_gray)
gray = cmap(np.arange(cmap.N))

cmap = plt.cm.get_cmap(cm_red)
red = cmap(np.arange(cmap.N))

cmap = plt.cm.get_cmap(cm_green)
green = cmap(np.arange(cmap.N))

cmap = plt.cm.get_cmap(cm_blue)
blue = cmap(np.arange(cmap.N))

fig, ax = plt.subplots(4, figsize=(10, 5),subplot_kw=dict(xticks=[], yticks=[]))
ax[0].imshow([gray], extent=[0, 10, 0, 1])
ax[1].imshow([red], extent=[0, 10, 0, 1])
ax[2].imshow([green], extent=[0, 10, 0, 1])
ax[3].imshow([blue], extent=[0, 10, 0, 1])


## 3.4 Função para separar imagem nos seus componentes R, G, B

In [ ]:
"""
Separar uma imagem RGB nos seus componentes
"""
def separate_components(img):
    r = img[:, :, 0]
    g = img[:, :, 1]
    b = img[:, :, 2]
    
    return r, g, b

## 3.5 Visualização da imagem nos seus componentes R, G, B 

In [ ]:
r, g, b = separate_components(img)
show_image(r, cm_red)
show_image(g, cm_green)
show_image(b, cm_blue)

In [ ]:
"""
Juntar as coponentes R, G e B para formar uma imagem
"""
def join_components(r, g, b):
    return np.dstack((r, g, b))

In [ ]:
img = join_components(r, g, b)
show_image(img)

## 4.1 Função para padding da imagem

In [ ]:
"""
Recebe uma imagem e altera as suas dimensões (m,n) para (16*p, 16*q).
Isto é realizado através da cópia da ultima coluna/linha até atingir o valor multiplo de 16.
Conta o numero de linhas/colunas adicionadas, (x,y) : 0<=x,y<=15.
Devolve um tuplo com as dimensoes incrementadas (x,y) e a imagem com as novas dimensoes (16*p, 16*q)
"""
def padding(img : np.array):
    img = img.copy()
    shape = img.shape
    
    x,y = (16-img.shape[0]%16)%16, (16-img.shape[1]%16)%16
    h_padding = np.repeat(img[-1:,:,:], x, axis = 0)
    img = np.concatenate((img, h_padding), axis = 0)

    v_padding = np.repeat(img[:,-1:,:], y, axis = 1)
    img = np.concatenate((img, v_padding), axis = 1)
    return shape, img

In [ ]:
"""
Recebe uma imagem e as dimensoes originais dela.
Faz slice da imagem, elimando todos os elementos incrementados no padding;
Devolve a imagem original
"""
def unpad(img : np.array, shape):
    img = img.copy()
    x,y,z = shape
    img = img[:x, :y]
    return img

In [ ]:
show_image(img)
shape, pad_img = padding(img)
show_image(pad_img)
unpad_img = unpad(img, shape)
show_image(unpad_img)

# Verificação que fica igual ao original
print(np.all(np.equal(img, unpad_img)))

print(f"Original shape: {img.shape}")
print(f"Padding shape: {pad_img.shape}")
print(f"After Padding and Unpadding shape: {unpad_img.shape}")

## 5.1 Função de conversão RGB para YCbCr

In [ ]:
"""
Converte imagem no formato RGB para imagem no formato yCbCr;
"""
def rgb_to_ycbcr(img : np.array):
    img.copy()
    
    y_cb_cr_mat = np.array([ [0.299    , 0.587    , 0.114    ]
                            ,[-0.168736, -0.331264, 0.5      ]
                            ,[0.5      , -0.418688, -0.081312] ])
    
    y  = y_cb_cr_mat[0,0] * img[:,:,0] + y_cb_cr_mat[0,1] * img[:,:,1] + y_cb_cr_mat[0,2]*img[:,:,2]
    cb = y_cb_cr_mat[1,0] * img[:,:,0] + y_cb_cr_mat[1,1] * img[:,:,1] + y_cb_cr_mat[1,2]*img[:,:,2] + 128
    cr = y_cb_cr_mat[2,0] * img[:,:,0] + y_cb_cr_mat[2,1] * img[:,:,1] + y_cb_cr_mat[2,2]*img[:,:,2] + 128
    
    y_cb_cr = np.dstack((y, cb, cr))
    return y_cb_cr

In [ ]:
"""
Converte imagem no formato YCbCr para imagem no formato RGB
"""
def ycbcr_to_rgb(img : np.array):
    img = img.copy()
    
    y_cb_cr_mat_inv = np.linalg.inv(
                                np.array([ [0.299    , 0.587    , 0.114    ]
                                        ,  [-0.168736, -0.331264, 0.5      ]
                                        ,  [0.5      , -0.418688, -0.081312] ])
                                    )
    y = img[:, :, 0]
    cb = img[:, :, 1] - 128
    cr = img[:, :, 2] - 128
    
    r = y + y_cb_cr_mat_inv[0,2]*cr
    g = y + y_cb_cr_mat_inv[1,1]*cb + y_cb_cr_mat_inv[1,2]*cr
    b = y + y_cb_cr_mat_inv[2,1]*cb
    
    rgb = np.dstack((r,g,b))
    rgb = np.round(rgb)
    rgb[rgb > 255] = 255
    rgb[rgb < 0] = 0
    
    return rgb.astype(np.uint8)

In [ ]:
ycbcr = rgb_to_ycbcr(pad_img)
rgb = ycbcr_to_rgb(ycbcr)

# Verificação que fica igual ao original
#print(np.all(np.equal(img, rgb)))

show_image(img)
show_image(rgb)

In [ ]:
# Método do colormap para representar os canais Cb e Cr:
# https://stackoverflow.com/questions/28638848/displaying-y-cb-and-cr-components-in-matlab

cm_cb = clr.LinearSegmentedColormap.from_list('cb', [(0.5, 0.5, 0), (0.5, 0.5, 1)], N = 256)
cm_cr = clr.LinearSegmentedColormap.from_list('cr', [(0, 0.5, 0.5), (1, 0.5, 0.5)], N = 256)

cmap = plt.cm.get_cmap(cm_cb)
cm_cb_rep = cmap(np.arange(cmap.N))

cmap = plt.cm.get_cmap(cm_cr)
cm_cr_rep = cmap(np.arange(cmap.N))

fig, ax = plt.subplots(2, figsize=(10, 2),subplot_kw=dict(xticks=[], yticks=[]))
ax[0].imshow([cm_cb_rep], extent=[0, 10, 0, 1])
ax[1].imshow([cm_cr_rep], extent=[0, 10, 0, 1])

In [ ]:
y, cb, cr = separate_components(ycbcr)

show_image(y, cm_gray)
show_image(cb, cm_cb)
show_image(cr, cm_cr)

> # Semana 2
Aplicação dos primeiros passos destrutivos do CODEC jpeg
> - *Downsample* com os rácios 4:2:2 e 4:2:0 
> - DCT aplicada á imagem toda

## 6.1 Downsample

In [ ]:
"""

"""
def sub_sample(y, cb, cr, downsample_ratio):
    y = y.copy()
    cb = cb.copy()
    cr = cr.copy()
    if downsample_ratio[-1] == 0:
        ratio = round(downsample_ratio[0]/downsample_ratio[1])
        cb = cb[::ratio,::ratio]
        cr = cr[::ratio,::ratio]
    else:
        v_ratio = round(downsample_ratio[0]/downsample_ratio[1])
        h_ratio = round(downsample_ratio[0]/downsample_ratio[2])
        cb = cb[:, ::v_ratio]
        cr = cr[:, ::v_ratio]
    return y,cb,cr

In [ ]:
"""
funciona em -- (4,2,0) e (4,2,2) --
Parametros:
            1. y --------------> matriz com os valores da luminancia Y
            2. cb -------------> matriz downsampled de cb
            3. cr -------------> matriz downsample de cr
            4. upsample_ratio -> tuplo com 3 inteiros correspondentes ao ratio de downsample (Y:Cr:Cb)
"""
def const_up_sample(y, cb, cr, upsample_ratio):
    y = y.copy()
    cb = cb.copy()
    cr = cr.copy()
    
    cbArr = np.zeros(shape = y.shape)
    crArr = np.zeros(shape = y.shape)
    
    if upsample_ratio[-1] == 0:
        ratio = int(upsample_ratio[0]/upsample_ratio[1])
        
        cbArr[::ratio,::ratio] = cb
        cbArr[1::ratio,1::ratio] = cb
        
        crArr[::ratio,::ratio] = cr
        crArr[1::ratio,1::ratio] = cr
        
    else:
        cb_ratio = int(upsample_ratio[0]/upsample_ratio[1])
        cr_ratio = int(upsample_ratio[0]/upsample_ratio[2])
        
        cbArr[:,::cb_ratio] = cb
        cbArr[:,1::cb_ratio] = cb
        
        crArr[:,::cr_ratio] = cr
        crArr[:,1::cr_ratio] = cr
        
    return y,cbArr,crArr

In [ ]:
"""
funciona em -- (4,2,0) e (4,2,2) --
Parametros:
            1. y --------------> matriz com os valores da luminancia Y
            2. cb -------------> matriz downsampled de cb
            3. cr -------------> matriz downsample de cr
            4. upsample_ratio -> tuplo com 3 inteiros correspondentes ao ratio de downsample (Y:Cr:Cb)
"""
def linear_up_sample(y, cb, cr, upsample_ratio):
    y = y.copy()
    cb = cb.copy()
    cr = cr.copy()
    
    new_cb = np.zeros(shape = y.shape)
    new_cr = np.zeros(shape = y.shape)
    
    if upsample_ratio[2] == 0:
        ratio = int(upsample_ratio[0]/upsample_ratio[1])
        #--------------------Cb interpolation----
        cb_cols_mean = (cb[:,:-1] + np.roll(cb, -1, 1)[:,:-1])//2
        cb_cols_mean = np.concatenate((cb_cols_mean, cb[:,-1:]), axis = 1)

        new_cb[::2,::2] = cb
        new_cb[::2,1::2] = cb_cols_mean

        cb = new_cb[::2]
        
        cb_lines_mean = (cb[:-1] + np.roll(cb, -1, 0)[:-1])//2
        cb_lines_mean = np.concatenate((cb_lines_mean, cb[-1:]), axis = 0)

        new_cb[1::2,:] = cb_lines_mean
        
        #------------------------Cr interpolation----
        cr_cols_mean = (cr[:,:-1] + np.roll(cr, -1, 1)[:,:-1])//2
        cr_cols_mean = np.concatenate((cr_cols_mean, cr[:,-1:]), axis = 1)

        new_cr[::2,::2] = cr
        new_cr[::2,1::2] = cr_cols_mean

        cr = new_cr[::2]
        
        cr_lines_mean = (cr[:-1] + np.roll(cr, -1, 0)[:-1])//2
        cr_lines_mean = np.concatenate((cr_lines_mean, cr[-1:]), axis = 0)

        new_cr[1::2,:] = cr_lines_mean
        
    else:
        cb_ratio = int(upsample_ratio[0]/upsample_ratio[1])
        cr_ratio = int(upsample_ratio[0]/upsample_ratio[2])
        
        cb_mean = (cb[:,:-1] + np.roll(cb, -1, 1)[:,:-1])/2
        cb_mean = np.concatenate((cb_mean, cb[:,-1:]), axis = 1)
        new_cb[:,::cb_ratio] = cb
        new_cb[:,1::cb_ratio] = cb_mean
        
        cr_mean = (cr[:,:-1] + np.roll(cr, -1, 1)[:,:-1])/2
        cr_mean = np.concatenate((cr_mean, cr[:,-1:]), axis = 1)
        new_cr[:,::cr_ratio] = cr
        new_cr[:,1::cr_ratio] = cr_mean
    
    return y,new_cb, new_cr

In [ ]:
#ratio = (4, 2, 2)
ratio = (4, 2, 0)
y_d, cb_d, cr_d = sub_sample(y, cb, cr, ratio)

# meter bloco para analise do downsample com os diferentes rácios

## 6.3 Análise da aplicação do *downsample*

dasdas dasd asdas das

## 7.1.1 DCT transformation

In [ ]:
def dct_(a):
    return dct(dct(a, norm="ortho").T, norm = "ortho").T

## 7.1.1 DCT reverse transformation

In [ ]:
def idct_(a_dct):
    return idct(idct(a_dct, norm='ortho').T, norm='ortho').T

In [ ]:
# DCT aplicada na imagem toda

y_dct = dct_(y_d)
cb_dct = dct_(cb_d)
cr_dct = dct_(cr_d)

show_image(np.log(np.abs(y_dct) + 0.0001), cm_gray)
show_image(np.log(np.abs(cb_dct) + 0.0001), cm_gray)
show_image(np.log(np.abs(cr_dct) + 0.0001), cm_gray)

In [ ]:
# Inverso da DCT

y_idct = idct_(y_dct)
cb_idct = idct_(cb_dct)
cr_idct = idct_(cr_dct)

show_image(y_idct, cm_gray)
show_image(cb_idct, cm_gray)
show_image(cr_idct, cm_gray)

In [ ]:
# Reconstrução com copia da coluna da esquerda

y, cb, cr = const_up_sample(y_idct, cb_idct, cr_idct, ratio)

show_image(y, cm_gray)
show_image(cb, cm_gray)
show_image(cr, cm_gray)

In [ ]:
#Recontrução com interpolação linear (média) da coluna anterior e seguinte

y, cb, cr = linear_up_sample(y_idct, cb_idct, cr_idct, ratio)

show_image(y, cm_gray)
show_image(cb, cm_gray)
show_image(cr, cm_gray)

> # Semana 3
Aplicação da DCT em blocos 8x8 ou 64x64
> - DCT em blocos e inverso

## 7.2.1 Dct in blocks

In [ ]:
def dct_n_blocks(y, cb, cr, step):
    dct_y = np.zeros(y.shape)
    dct_cb = np.zeros(cb.shape)
    dct_cr = np.zeros(cr.shape)
    
    for i in range(0, y.shape[0], step):
        for j in range(0, y.shape[1], step):
            dct_y[i:i+step, j:j+step] = dct_(y[i:i+step, j:j+step])
    
    for i in range(0, cb.shape[0], step):
        for j in range(0, cb.shape[1], step):
            dct_cb[i:i+step, j:j+step] = dct_(cb[i:i+step, j:j+step])
            dct_cr[i:i+step, j:j+step] = dct_(cr[i:i+step, j:j+step])
            
    return dct_y, dct_cb, dct_cr

## 7.2.1 Reverse DCT in blocks

In [ ]:
def idct_n_blocks(y, cb, cr, step):
    idct_y = np.zeros(y.shape)
    idct_cb = np.zeros(cb.shape)
    idct_cr = np.zeros(cr.shape)
    
    for i in range(0, y.shape[0], step):
        for j in range(0, y.shape[1], step):
            idct_y[i:i+step, j:j+step] = idct_(y[i:i+step, j:j+step])
    
    for i in range(0, cb.shape[0], step):
        for j in range(0, cb.shape[1], step):
            idct_cb[i:i+step, j:j+step] = idct_(cb[i:i+step, j:j+step])
            idct_cr[i:i+step, j:j+step] = idct_(cr[i:i+step, j:j+step])
            
    return idct_y, idct_cb, idct_cr

In [ ]:
# DCT em blocos

n = 8
y_dctn, cb_dctn, cr_dctn = dct_n_blocks(y, cb, cr, n)

show_image(np.log(np.abs(y_dctn) + 0.0001), cm_gray)
show_image(np.log(np.abs(cb_dctn) + 0.0001), cm_gray)
show_image(np.log(np.abs(cr_dctn) + 0.0001), cm_gray)

In [ ]:
# Inverter DCT em blocos

y_d, cb_d, cr_d = idct_n_blocks(y_dctn, cb_dctn, cr_dctn, n)

show_image(y_d, cm_gray)
show_image(cb_d, cm_gray)
show_image(cr_d, cm_gray)

> # Semana 4
Quantização da imagem e compressão dos coponentes DC da DCT com *delta encoding*
> - Quantização dos canais Y, Cb e Cr com as matrizes adequadas
> - *delta encoding* para as componentes DC da DCT.
>   - A compressão dos componentes AC não faz parte deste trabalho


## 8 Quantização

In [ ]:
def quantitization(y_dctn, cb_dctn, cr_dctn, fator_qualidade):
    Q_y = np.array(
        [
            [16, 11, 10, 16, 24, 40, 51, 61],
            [12, 12, 14, 19, 26, 58, 60, 55],
            [14, 13, 16, 24, 40, 57, 69, 56],
            [14, 17, 22, 29, 51, 87, 80, 62],
            [18, 22, 37, 56, 68, 109, 103, 77],
            [24, 35, 55, 64, 81, 104, 113, 92],
            [49, 64, 78, 87, 103, 121, 120, 101],
            [72, 92, 95, 98, 112, 100, 103, 99],
        ]
    )
    Q_CbCr = np.array(
        [
            [17, 18, 24, 47, 99, 99, 99, 99],
            [18, 21, 26, 66, 99, 99, 99, 99],
            [24, 26, 56, 99, 99, 99, 99, 99],
            [47, 66, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
        ]
    )
    
    y_t = y_dctn.copy()
    cb_t = cb_dctn.copy()
    cr_t = cr_dctn.copy()
    
    s = (100 - fator_qualidade)/50 if fator_qualidade >= 50 else 50/fator_qualidade
    Q_y[::,::] = np.round(s * Q_y) if s != 0 else 1
    Q_CbCr[::,::] = np.round(s * Q_CbCr) if s!=0 else 1
    
    Q_y[Q_y > 255] = 255
    Q_CbCr[Q_CbCr > 255] = 255
    
    for i in range(0, y_t.shape[0], 8):
        for j in range(0, y_t.shape[1], 8):
            y_t[i:i+8, j:j+8] = y_t[i:i+8, j:j+8] / Q_y
            
    for i in range(0, cb_t.shape[0], 8):
        for j in range(0, cb_t.shape[1], 8):
            cb_t[i:i+8, j:j+8] = cb_t[i:i+8, j:j+8] / Q_CbCr
            cr_t[i:i+8, j:j+8] = cr_t[i:i+8, j:j+8] / Q_CbCr
            
    y_t = np.round(y_t)
    cb_t = np.round(cb_t)
    cr_t = np.round(cr_t)
    return y_t, cb_t, cr_t


In [ ]:
def quantitization_inverse(y_dctn_q, cb_dctn_q, cr_dctn_q, fator_qualidade):
    Q_y = np.array(
        [
            [16, 11, 10, 16, 24, 40, 51, 61],
            [12, 12, 14, 19, 26, 58, 60, 55],
            [14, 13, 16, 24, 40, 57, 69, 56],
            [14, 17, 22, 29, 51, 87, 80, 62],
            [18, 22, 37, 56, 68, 109, 103, 77],
            [24, 35, 55, 64, 81, 104, 113, 92],
            [49, 64, 78, 87, 103, 121, 120, 101],
            [72, 92, 95, 98, 112, 100, 103, 99],
        ]
    )
    
    Q_CbCr = np.array(
        [
            [17, 18, 24, 47, 99, 99, 99, 99],
            [18, 21, 26, 66, 99, 99, 99, 99],
            [24, 26, 56, 99, 99, 99, 99, 99],
            [47, 66, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
        ]
    )
    y_t = y_dctn_q.copy()
    cb_t = cb_dctn_q.copy()
    cr_t = cr_dctn_q.copy()
    
    s = (100 - fator_qualidade)/50 if fator_qualidade >= 50 else 50/fator_qualidade
    Q_y[::,::] = np.round(s * Q_y) if s != 0 else 1
    Q_CbCr[::,::] = np.round(s * Q_CbCr) if s!=0 else 1
    
    Q_y[Q_y > 255] = 255
    Q_CbCr[Q_CbCr > 255] = 255
    
    for i in range(0, y_t.shape[0], 8):
        for j in range(0, y_t.shape[1], 8):
            y_t[i:i+8, j:j+8] = y_t[i:i+8, j:j+8] * Q_y
    
    for i in range(0, cb_t.shape[0], 8):
        for j in range(0, cb_t.shape[1], 8):
            cb_t[i:i+8, j:j+8] = cb_t[i:i+8, j:j+8] * Q_CbCr
            cr_t[i:i+8, j:j+8] = cr_t[i:i+8, j:j+8] * Q_CbCr
            
    y_t = np.round(y_t)
    cb_t = np.round(cb_t)
    cr_t = np.round(cr_t)
    return y_t, cb_t, cr_t

In [ ]:
# Fator de qualidade para a quantização da imagem

fator = 75
y_dctn_q, cb_dctn_q, cr_dctn_q = quantitization(y_dctn, cb_dctn, cr_dctn, fator)
y_dctn_qi, cb_dctn_qi, cr_dctn_qi = quantitization_inverse(y_dctn_q, cb_dctn_q, cr_dctn_q, fator)

In [ ]:
#show_image(y_dctn_q, cm_gray)
#show_image(y_dctn_qi, cm_gray)
#show_image(cb_dctn_q, cm_gray)
#show_image(cr_dctn_q, cm_gray)

## 8.2 DPCM

In [ ]:
# Delta encoding aplicado aos coeficientes DC da DCT em blocos

def diff(bloco):
    x, y = bloco.shape
    bloco = bloco.flatten()
    bloco[1:] -= bloco[:-1]
    return bloco.reshape(x,y)

def DPCM(y,cb,cr):

    yBlocks = y.copy()
    cbBlocks = cb.copy()
    crBlocks = cr.copy()
    
    yBlocks[::8,::8] = diff(y[::8,::8])
    cbBlocks[::8,::8] = diff(cb[::8,::8])
    crBlocks[::8,::8] = diff(cr[::8,::8])
    
        
    return yBlocks, cbBlocks, crBlocks

In [ ]:
def diff_reverse(bloco):
    x, y = bloco.shape
    bloco = bloco.flatten()
    for i in range(1,bloco.shape[0]):
        bloco[i] = bloco[i] + bloco[i-1]
    return bloco.reshape(x,y)

In [ ]:
def DPCM_reverse(yBlocks, cbBlocks, crBlocks):
    y = yBlocks.copy()
    cr = crBlocks.copy()
    cb = crBlocks.copy()
    
    y[::8,::8] = diff_reverse(yBlocks[::8,::8])
    cr[::8,::8] = diff_reverse(crBlocks[::8,::8])
    cb[::8,::8] = diff_reverse(cbBlocks[::8,::8])
        
    return y, cb, cr

In [ ]:
d_y, d_cb, d_cr = DPCM(y_dctn_q, cb_dctn_q, cr_dctn_q)
id_y, id_cb, id_cr = DPCM_reverse(d_y, d_cb, d_cr)

> # Semana 5
Aplicação de todas as funções criadas antriormente com os fatores de qualidade 10, 25, 50, 75, 100
> - Cálculo das métricas de  distorção (MSE, RMSE, SNR, PSNR) para todas as imagens e fatores de qualidade


In [ ]:
def MSE(orig, compressed):
    return np.sum(np.square(orig-compressed))/np.prod(orig.shape)

In [ ]:
def RMSE(orig, compressed):
    return np.sqrt(MSE(orig,compressed))

In [ ]:
def SNR(orig, compressed):
    P = np.sum(np.square(orig))/np.prod(orig.shape)
    return 10*np.log10(P/MSE(orig, compressed))

In [ ]:
def PSNR(orig, compressed):
    return 10*np.log10(np.square(orig).max()/MSE(orig,compressed))

## Encoder and decoder functions

In [ ]:
def encoder(filename, ratio, n, factor):
    img = read_image(filename)
    
    shape, pad_img = padding(img)
    
    ycbcr = rgb_to_ycbcr(pad_img)
    
    y, cb, cr = separate_components(ycbcr)
    
    y_d, cb_d, cr_d = sub_sample(y, cb, cr, ratio)
    
    y_dct, cb_dct, cr_dct = dct_n_blocks(y_d, cb_d, cr_d, n)
    
    y_dct_q, cb_dct_q, cr_dct_q = quantitization(y_dct, cb_dct, cr_dct, factor)
    
    delta_y, delta_cb, delta_cr = DPCM(y_dct_q, cb_dct_q, cr_dct_q)

    return delta_y, delta_cb, delta_cr, shape

In [ ]:
def decoder(delta_y, delta_cb, delta_cr, ratio, shape, n, factor):
    y_dct_q, cb_dct_q, cr_dct_q = DPCM_reverse(delta_y, delta_cb, delta_cr)
    
    y_dct, cb_dct, cr_dct = quantitization_inverse(y_dct_q, cb_dct_q, cr_dct_q, factor)
    
    y_idct, cb_idct, cr_idct = idct_n_blocks(y_dct, cb_dct, cr_dct, n) # DCT em blocos n x n

    #y, cb, cr = const_up_sample(y_idct, cb_idct, cr_idct, ratio) # Upsample: cópia da coluna á esquerda
    y, cb, cr = linear_up_sample(y_idct, cb_idct, cr_idct, ratio) # Upsample: média entre o anterior e o seguinte
    
    ycbcr = join_components(y, cb, cr)
    
    img_rgb = ycbcr_to_rgb(ycbcr)
    
    img = unpad(img_rgb, shape)
    
    return img

In [ ]:
# Encode
ratio = (4, 2, 2)
n = 8
fator = 75
y_dct, cb_dct, cr_dct, shape = encoder('imagens/barn_mountains.bmp', ratio, n, fator)

# Decode
img = decoder(y_dct, cb_dct, cr_dct, ratio, shape, n, 75)
show_image(img)
print(MSE(im.imread('imagens/barn_mountains.bmp'), img),
     RMSE(im.imread('imagens/barn_mountains.bmp'), img),
     SNR(im.imread('imagens/barn_mountains.bmp'), img),
     PSNR(im.imread('imagens/barn_mountains.bmp'), img),
     sep = "\n")